In [1]:
import numpy as np
import pandas as pd
from random import sample, seed
from azure.ai.translation.text.models import InputTextItem
from azure.ai.translation.text import TextTranslationClient, TranslatorCredential

seed(42)

endpoint = "<azure-translation-endpoint>"
subscription_key = "<azure-translation-subscription-key>"
location = "centralindia"

text_translator = TextTranslationClient(
    endpoint=endpoint, credential=TranslatorCredential(subscription_key, location)
)

In [2]:
def translate(from_lang, to_lang, input_string):
    source_language = from_lang
    target_languages = [to_lang]
    input_text_elements = [InputTextItem(text=input_string)]

    response = text_translator.translate(
        content=input_text_elements,
        to=target_languages,
        from_parameter=source_language,
    )
    translation = response[0]
    for translated_text in translation.translations:
        return translated_text.text

In [3]:
data = np.load("CORGI-PC_splitted_biased_corpus_v1.npy", allow_pickle=True).item()
data_sampled = sample(data["test"]["ori_sentence"].tolist(), k=100)
data_sampled_translated = [translate("zh", "hi", sentence) for sentence in data_sampled]
    
df = pd.DataFrame({"source": data_sampled, "translation": data_sampled_translated})
df.to_excel("CORGI_PM_sampled_translated.xlsx", index=False)

In [5]:
df_ = pd.read_csv("full_BUG.csv")
mask = df_["stereotype"] == 1
df_ = df_[mask].sample(n=100)

data_sampled = df_["sentence_text"].tolist()
data_sampled_translated = [translate("en", "hi", sentence) for sentence in data_sampled]

df = pd.DataFrame({"source": data_sampled, "translation": data_sampled_translated})
df.to_excel("full_BUG_sampled_translated.xlsx", index=False)